In [52]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


In [51]:
def Kulback_Leibler(q,p): 
    return q*np.log(q/p)

In [6]:


def train(models, q , p_0,   n_iter=100, lr=1e-2):
    nombre_de_neurones=len(models)
    optimizers=[torch.optim.Adam(models[i].parameters(), lr=lr) for i in range(nombre_de_neurones)]
    p=p_0
    for epoch in range(n_iter):
        p='...'# à compléter
        for i in range(nombre_de_neurones):
            optimizers[i].zero_grad()
            loss=Kulback_Leibler(p, q)
            loss.backward()
            optimizers[i]
           


In [ ]:

def train(model, n_iter=100, lr=1e-2):
    plt.figure(figsize=(20, 8))
    axs = [plt.subplot(2, 5, i) for i in range(1, 11)]
    losses = []
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # What is Adam? Which other simpler optimizers can you use?

    for epoch in range(n_iter):
        optimizer.zero_grad() # What is this step? IMPORTANT LINE
        loss = loss_fn(model(x_train), y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
       
        if epoch % (n_iter/10) == 0:
            print(f'Epoch {epoch}: {loss.item()}')

            # plot progress
            ax_index = int(epoch // (n_iter/10))
            plt.sca(axs[ax_index])
            plt.plot(x_grid, target(x_grid), label='target')
            plt.plot(x_grid, grab(model(x_grid)).squeeze(), label='model init')
            
    return losses




def loss_fn(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

In [39]:
n=4
M = torch.zeros((n, n), dtype=torch.int)  # Crée une matrice de zéros de taille n x n
        
for i in range(n):
    for j in range(i, n):
        M[i][j] = 0

# Remplit la partie inférieure de la diagonale avec des uns
for i in range(1, n):
        for j in range(i):
                M[i][j] = 1

In [44]:
A=torch.tensor([[1, 2], [3, 4]])

In [48]:
class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, mask, bias=True):
        super(MaskedLinear, self).__init__(in_features, out_features, bias)
        self.mask = mask

    def forward(self, input):
        return nn.functional.linear(input, self.weight * self.mask, self.bias)

In [54]:
torch.sigmoid(torch.tensor([0.0, 2.0]))

tensor([0.5000, 0.8808])

In [50]:
class VAN(nn.Module):
    def __init__(self, input_size, activation=torch.sigmoid):
        super(VAN, self).__init__() #initialisation obligatoire
        self.input_size = input_size

        # Création de la matrice de masque : que des 0 sur et au dessus de la diagonale et que des 1 dessous
        M = torch.zeros((input_size, input_size), dtype=torch.int)
        for i in range(input_size):
            for j in range(i, input_size):
                M[i][j] = 0
        for i in range(1, input_size):
            for j in range(i):
                M[i][j] = 1

        self.fc1 = MaskedLinear(input_size, input_size, mask=M)
        self.activation = activation

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        # à cette ligne on a multiplié x par la matrice de masque (triangulaire inférieure), puis appliqué la fonction d'activation
        # donc la première coordonnée de x vaut activation(0) (normal, s^_1 ne dépend de personne)
        # il faut donc ajouter 0.5 à la première coordonnée pour montrer qu'elle vaut 0 et 1 avec proba 0.5
        x[0] = 0.5
        return x


In [81]:
torch.sigmoid(torch.tensor([0.5, 2.0]))


tensor([0.6225, 0.8808])

In [49]:
n=7
M = torch.zeros((n, n), dtype=torch.int)  # Crée une matrice de zéros de taille n x n

for i in range(n):
    for j in range(i, n):
        M[i][j] = 0

# Remplit la partie inférieure de la diagonale avec des uns
for i in range(1, n):
        for j in range(i):
                M[i][j] = 1


In [ ]:

class MLP(nn.Module):
    """
    a MLP class inheriting from the parent class nn.Module. 
    nn.Module is the generic parent class of models in Pytorch.
    It requires a method called forward. 
    
    Pytorch will be able to recursively recover all parameters of the
    attributes of a nn.Module object provided the attributes have themselves
    type nn.Modules or nn.ModuleList.

    In this implementation we explicitly specify the scale of initialization 
    of the weight matrices.
    """
    def __init__(self, layerdims, activation=torch.relu, out_activation=None, init_scale=1):
        super(MLP, self).__init__()

        self.layerdims = layerdims
        self.activation = activation
        self.out_activation = out_activation

        linears = [
            nn.Linear(layerdims[i], layerdims[i + 1]) for i in range(len(layerdims) - 1)
        ]

        if init_scale is not None:
            for l, layer in enumerate(linears):
                torch.nn.init.normal_(
                    layer.weight, std=init_scale / np.sqrt(layerdims[l])
                )
                torch.nn.init.zeros_(layer.bias)

        self.linears = nn.ModuleList(linears)

    def forward(self, x):
        layers = list(enumerate(self.linears))
        for _, l in layers[:-1]:
            x = self.activation(l(x))
        y = layers[-1][1](x)
        if self.out_activation is not None:
            y = self.out_activation(y)
        return y



In [20]:
l=[1, 2, 3, 4]
l[:-1]

[1, 2, 3]

In [ ]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
output.shape

torch.Size([128, 30])

In [ ]:
m.weight[0]

tensor([ 0.0392,  0.0495, -0.1220, -0.0829, -0.1599, -0.1073,  0.1244, -0.0429,
         0.2159, -0.1070,  0.1197,  0.0551, -0.1100,  0.1403, -0.2043, -0.0713,
        -0.0352,  0.1676,  0.0583, -0.1296], grad_fn=<SelectBackward0>)